<a href="https://colab.research.google.com/github/coldsober-irene/NLP-fellowship/blob/main/Copy_of_hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# REQUIRED IMPORTS
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np

In [68]:
# DATA MINING
class dataMining:
  def __init__(self, jobSource = "Job In Rwanda"):
    self.source = jobSource
    self.jobIdentification = {} # {"software_developer":[published_date, company, requirements, deadline, full_description],....}
  
  # SOUP
  def Soup(self, url):
    page = requests.get(url).content
    soup = bs(page, "html.parser")
    return soup

  def get_links(self):
    # GET SOUP OBJECT
    soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    # GET ALL JOBS LINKS
    company_infoLink = []
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] else company_infoLink.append("https://www.jobinrwanda.com"+link['href']) for div in soup.find_all("div", class_= "card-body p-2") for link in div.find_all("a")]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  def get_jobInfo(self):
    jobLinks = self.get_links()[0]
    info = {}
    # REFINE THE INFO INTO A DICTIONARY
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      # OTHER DETAILS EXCEPT THE ONE ON POSITION 0
      otherInfoPattern = re.compile(":")
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          # print(app_link)
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except AttributeError:
            pass
        all_text[index] = temp

    return all_text


In [73]:
jobInRwanda = dataMining()
job_text = jobInRwanda.get_description()
info = jobInRwanda.get_jobInfo()

In [ ]:
info[10].values()

In [ ]:
print(job_text[2])

In [ ]:
# MERGE JOB DESCRIPTION WITH THE JOB INFO
for index in list(info.keys()):
  info_value = list(info[index].values())
  try:
    info[index] = info_value.append(job_text[index][0]) # APPEND APPLICATION LINK
    # job_text[index] = job_text[index].append(job_text[index][1:]) # JOB DESCRIPTION
  except (AttributeError, TypeError):
    print(job_text[index], index)

In [75]:
print(job_text[10])

None


In [24]:
# for infos in info[5]:
#   print(infos)
# all_text[50]
info[40]

{'views': '1751 times',
 'Location': ' , RW',
 'Sector': ' Administration, Business, Law, Other',
 'Education level ': ' Bachelor',
 'Desired experience': ' Entry level (1 to 3 years of experience)',
 'Contract type ': ' Full-time',
 'Deadline': ' Tuesday, 15/11/2022 17',
 'Number of positions': ' 1'}

In [15]:
pat = re.compile("\d+.*")
pat.search(info[5][0]).group()

'1341 times'

In [4]:
# DATA SORTING

In [5]:
# DATA PRESENTATION